<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Access-Token" data-toc-modified-id="Load-Access-Token-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Access Token</a></span></li><li><span><a href="#Query-Quizzes-in-One-Course" data-toc-modified-id="Query-Quizzes-in-One-Course-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Query Quizzes in One Course</a></span></li><li><span><a href="#Get-One-Quiz-Report" data-toc-modified-id="Get-One-Quiz-Report-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get One Quiz Report</a></span></li><li><span><a href="#Query-Questions-for-One-Quiz" data-toc-modified-id="Query-Questions-for-One-Quiz-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Query Questions for One Quiz</a></span></li><li><span><a href="#Query-Submissions-for-One-Quiz" data-toc-modified-id="Query-Submissions-for-One-Quiz-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Query Submissions for One Quiz</a></span></li><li><span><a href="#Quiz-Submission-Questions" data-toc-modified-id="Quiz-Submission-Questions-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Quiz Submission Questions</a></span></li></ul></div>

In [140]:
import requests
import os
import json
import csv
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from html.parser import HTMLParser
import re, cgi
from lxml import html

# Load Access Token

In [35]:
with open('.secrets/creds.json') as f:
    creds = json.load(f)

In [36]:
ACCESS_TOKEN = creds['key']

In [37]:
type(creds)

dict

# Query Quizzes in One Course

In [78]:
api = 'https://learning.flatironschool.com/api/v1/courses'
course_id = 4142
endpoint = '/{}/quizzes'.format(course_id)

url = api + endpoint
print(url)

headers = {
    'Authorization': 'Bearer ' + ACCESS_TOKEN
}

url_params = {
    'search_term': 'Exit Ticket',
    'offset': 0
}
response = requests.get(url, headers=headers, params=url_params)
print(response.status_code)

https://learning.flatironschool.com/api/v1/courses/4142/quizzes
200


In [94]:
quizzes = pd.DataFrame(response.json())
quizzes.head()

,id,title,html_url,mobile_url,description,quiz_type,time_limit,timer_autosubmit_disabled,shuffle_answers,show_correct_answers,...,quiz_submission_versions_html_url,assignment_id,one_time_results,only_visible_to_overrides,assignment_group_id,show_correct_answers_last_attempt,version_number,has_access_code,post_to_sis,migration_id
0,15563,Aggregating and Combining DataFrames Exit Ticket,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/courses/41...,,practice_quiz,None,False,False,True,...,https://learning.flatironschool.com/courses/41...,None,False,False,8290,False,7,False,None,deletedsub_1752_mastercourse_15802_36_970c5757...
1,15589,APIs Exit Ticket,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/courses/41...,,practice_quiz,None,False,False,True,...,https://learning.flatironschool.com/courses/41...,None,False,False,8290,False,7,False,None,deletedsub_1752_mastercourse_15802_36_5fb2a61b...
2,15583,Bash Exit Ticket,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/courses/41...,,practice_quiz,None,False,False,True,...,https://learning.flatironschool.com/courses/41...,None,False,False,8290,False,3,False,None,deletedsub_1752_mastercourse_15802_36_2ed5bee9...
3,15557,Collaborating with Git Exit Ticket,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/courses/41...,,practice_quiz,None,False,False,True,...,https://learning.flatironschool.com/courses/41...,None,False,False,8290,False,7,False,None,deletedsub_1752_mastercourse_15802_36_da657e1e...
4,15566,Data Analysis with CSVs Exit Ticket,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/courses/41...,,practice_quiz,None,False,False,True,...,https://learning.flatironschool.com/courses/41...,None,False,False,8290,False,5,False,None,deletedsub_1752_mastercourse_15802_36_18ce7672...


In [96]:
exit_ticket_quiz_info = quizzes[['id', 'title', 'html_url', 'quiz_reports_url', 'quiz_statistics_url']]
exit_ticket_quiz_info.head()

,id,title,html_url,quiz_reports_url,quiz_statistics_url
0,15563,Aggregating and Combining DataFrames Exit Ticket,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/api/v1/cou...,https://learning.flatironschool.com/api/v1/cou...
1,15589,APIs Exit Ticket,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/api/v1/cou...,https://learning.flatironschool.com/api/v1/cou...
2,15583,Bash Exit Ticket,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/api/v1/cou...,https://learning.flatironschool.com/api/v1/cou...
3,15557,Collaborating with Git Exit Ticket,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/api/v1/cou...,https://learning.flatironschool.com/api/v1/cou...
4,15566,Data Analysis with CSVs Exit Ticket,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/api/v1/cou...,https://learning.flatironschool.com/api/v1/cou...


In [98]:
exit_ticket_quiz_info['quiz_reports_url'][0]

'https://learning.flatironschool.com/api/v1/courses/4142/quizzes/15563/reports'

# Get One Quiz Report

In [124]:
# Generate reports
api = 'https://learning.flatironschool.com/api/v1/courses'
course_id = 4142
quiz_id = 15589
endpoint = '/{}/quizzes/{}/reports'.format(course_id, quiz_id)

url = api + endpoint
print(url)

headers = {
    'Authorization': 'Bearer ' + ACCESS_TOKEN
}

url_params = {
    'quiz_report[report_type]': 'student_analysis',
}
response = requests.post(url, headers=headers, params=url_params)
print(response.status_code)

https://learning.flatironschool.com/api/v1/courses/4142/quizzes/15589/reports
200


In [126]:
# Need to wait and resend request to get file once report is generated
api = 'https://learning.flatironschool.com/api/v1/courses'
course_id = 4142
quiz_id = 15589
endpoint = '/{}/quizzes/{}/reports'.format(course_id, quiz_id)

url = api + endpoint
print(url)

headers = {
    'Authorization': 'Bearer ' + ACCESS_TOKEN
}

url_params = {
    'quiz_report[report_type]': 'student_analysis',
}
response = requests.post(url, headers=headers, params=url_params)
print(response.status_code)

https://learning.flatironschool.com/api/v1/courses/4142/quizzes/15589/reports
200


In [127]:
response.json()

{'id': 1338,
 'report_type': 'student_analysis',
 'readable_type': 'Student Analysis',
 'includes_all_versions': False,
 'includes_sis_ids': True,
 'generatable': True,
 'anonymous': False,
 'url': 'https://learning.flatironschool.com/api/v1/courses/4142/quizzes/15589/reports/1338',
 'progress_url': 'https://learning.flatironschool.com/api/v1/progress/31212',
 'created_at': '2021-05-11T14:01:10Z',
 'updated_at': '2021-05-11T14:01:23Z',
 'file': {'id': 1596768,
  'uuid': 'rvtf0KE0JdjtHoMWKMZknSm0EflvQLaO7OaXopqE',
  'folder_id': None,
  'display_name': 'APIs Exit Ticket Quiz Student Analysis Report.csv',
  'filename': 'quiz_student_analysis_report.csv',
  'upload_status': 'success',
  'content-type': 'text/csv',
  'url': 'https://learning.flatironschool.com/files/1596768/download?download_frd=1&verifier=rvtf0KE0JdjtHoMWKMZknSm0EflvQLaO7OaXopqE',
  'size': 2614,
  'created_at': '2021-05-11T14:01:23Z',
  'updated_at': '2021-05-11T14:01:23Z',
  'unlock_at': None,
  'locked': False,
  'hidd

In [130]:
student_report_url = response.json()['file']['url']

In [135]:
student_report = requests.get(student_report_url)
print(student_report.status_code)

200


In [179]:
csv_reader = csv.reader(student_report.content.decode('utf-8').splitlines())
csv_reader_list = list(csv_reader)

In [180]:
# Clean column names
column_names = csv_reader_list[0]
for i in range(len(column_names)):
    if column_names[i] == '1.0':
        column_names[i] = column_names[i-1].split(':')[0] + " Points"

In [181]:
column_names

['name',
 'id',
 'sis_id',
 'section',
 'section_id',
 'section_sis_id',
 'submitted',
 '103049: Which of the following data formats am I most likely to get back from an API request?',
 '103049 Points',
 '103050: What is the purpose of a status code (like 200 or 404)?',
 '103050 Points',
 "103051: What does a '?' in a URL indicate?",
 '103051 Points',
 '99340: This lecture was...',
 '99340 Points',
 '99341: This lecture was...',
 '99341 Points',
 '99342: What did you like best about this lecture?',
 '99342 Points',
 '99343: Do you have any thoughts about how this lecture might be improved?',
 '99343 Points',
 '99344: Is there any other feedback you have about this lecture?',
 '99344 Points',
 'n correct',
 'n incorrect',
 'score']

In [178]:
student_report_df = pd.DataFrame(data = csv_reader_list[1:], columns = column_names)
student_report_df.head()

,name,id,sis_id,section,section_id,section_sis_id,submitted,103049: Which of the following data formats am I most likely to get back from an API request?,103049 Points,103050: What is the purpose of a status code (like 200 or 404)?,...,99341 Points,99342: What did you like best about this lecture?,99342 Points,99343: Do you have any thoughts about how this lecture might be improved?,99343 Points,99344: Is there any other feedback you have about this lecture?,99344 Points,n correct,n incorrect,score
0,David Shirley,5438,d1dd1a65-4608-4956-a0b4-81fc8bfb47e1,Phase 1,2878,,2021-05-07 13:09:03 UTC,json,1.0,To give an indication of the status of the con...,...,1.0,Was a good and helpful Lecture about APIs,0.0,"Was a good lecture, no need for improvement.",0.0,N/A,0.0,8,0,5.0
1,James Hodgens,5655,28970fdc-5c7a-43ac-b1ea-c2913f77d4c4,Phase 1,2878,,2021-05-06 22:59:16 UTC,json,1.0,To give an indication of the status of the con...,...,1.0,The Yelp exercise,0.0,No,0.0,No,0.0,8,0,5.0
2,Vaneeza Ahmad,5073,1f2bb521-ea4e-4bbe-9cd9-e2c03c966a79,Washington D.C.,3224,,2021-05-06 19:56:42 UTC,json,1.0,To give an indication of the status of the con...,...,0.0,The code examples were helpful.,0.0,APIs could have been explained in a little mor...,0.0,It could have been more engaging.,0.0,6,2,3.0
3,Benjamin McCarty,5526,dcabb823-cbf0-43f4-a0fd-237c30f29e0a,Washington D.C.,3224,,2021-05-06 19:49:28 UTC,json,1.0,To give an indication of the status of the con...,...,0.0,,0.0,,0.0,,0.0,6,2,3.0
4,Adam Cumurcu,5283,c3cb7cb3-803a-450b-96d9-a1d59e4e510c,Washington D.C.,3224,,2021-05-06 19:48:01 UTC,json,1.0,To give an indication of the status of the con...,...,1.0,It provided a good overview.,0.0,No,0.0,No,0.0,8,0,5.0


# Query Questions for One Quiz

In [72]:
api = 'https://learning.flatironschool.com/api/v1'
course_id = 4142
quiz_id = 15589
endpoint = '/courses/{}/quizzes/{}/questions'.format(course_id, quiz_id)

url = api + endpoint
print(url)

headers = {
    'Authorization': 'Bearer ' + ACCESS_TOKEN
}

url_params = {
    'offset': 0
}
response = requests.get(url, headers=headers, params=url_params)
print(response.status_code)

https://learning.flatironschool.com/api/v1/courses/4142/quizzes/15589/questions
200


In [74]:
response.json()

[{'id': 103049,
  'quiz_id': 15589,
  'quiz_group_id': None,
  'assessment_question_id': None,
  'position': 1,
  'question_name': 'Question 1',
  'question_type': 'multiple_choice_question',
  'question_text': '<span style="font-size: 18pt;">Which of the following data formats am I most likely to get back from an API request?</span>',
  'points_possible': 1.0,
  'correct_comments': '',
  'incorrect_comments': '',
  'neutral_comments': None,
  'correct_comments_html': None,
  'incorrect_comments_html': None,
  'neutral_comments_html': None,
  'answers': [{'weight': 0,
    'migration_id': 'RESPONSE_8305',
    'id': 8305,
    'text': 'html'},
   {'weight': 0, 'migration_id': 'RESPONSE_5312', 'id': 5312, 'text': 'csv'},
   {'weight': 0, 'migration_id': 'RESPONSE_3575', 'id': 3575, 'text': 'tsv'},
   {'weight': 0,
    'migration_id': 'RESPONSE_1758',
    'id': 1758,
    'text': 'data frame'},
   {'weight': 100,
    'migration_id': 'RESPONSE_9903',
    'id': 9903,
    'text': 'json'}],
  'v

In [73]:
quiz_questions = pd.DataFrame(response.json())
quiz_questions

,id,quiz_id,quiz_group_id,assessment_question_id,position,question_name,question_type,question_text,points_possible,correct_comments,...,correct_comments_html,incorrect_comments_html,neutral_comments_html,answers,variables,formulas,answer_tolerance,formula_decimal_places,matches,matching_answer_incorrect_matches
0,103049,15589,None,NaN,1,Question 1,multiple_choice_question,"<span style=""font-size: 18pt;"">Which of the fo...",1.0,,...,None,None,None,"[{'weight': 0, 'migration_id': 'RESPONSE_8305'...",None,None,None,None,None,None
1,103050,15589,None,116679.0,2,Question 2,multiple_choice_question,"<span style=""font-size: 18pt;"">What is the pur...",1.0,,...,None,None,None,"[{'weight': 100, 'migration_id': 'RESPONSE_246...",None,None,None,None,None,None
2,103051,15589,None,116697.0,3,Question 3,multiple_choice_question,"<span style=""font-size: 18pt;"">What does a '?'...",1.0,,...,None,None,None,"[{'weight': 0, 'migration_id': 'RESPONSE_2000'...",None,None,None,None,None,None
3,99340,15589,None,114667.0,4,Question 4,multiple_choice_question,"<span style=""font-size: 18pt;"">This lecture wa...",1.0,,...,None,None,None,"[{'weight': 100, 'migration_id': 'RESPONSE_468...",None,None,None,None,None,None
4,99341,15589,None,114668.0,5,Question 5,multiple_choice_question,"<span style=""font-size: 18pt;"">This lecture wa...",1.0,,...,None,None,None,"[{'weight': 100, 'migration_id': 'RESPONSE_579...",None,None,None,None,None,None
5,99342,15589,None,114669.0,6,Question 6,essay_question,"<span style="" font-size: 18pt;"">What did you l...",1.0,,...,None,None,None,[],None,None,None,None,None,None
6,99343,15589,None,114671.0,7,Question 7,essay_question,"<span style=""font-size: 18pt;""><span>Do you ha...",1.0,,...,None,None,None,[],None,None,None,None,None,None
7,99344,15589,None,114666.0,8,Question 8,essay_question,"<span style=""font-size: 18pt;""><span>Is there ...",1.0,,...,None,None,None,[],None,None,None,None,None,None


In [90]:
# Parse HTML from quiz questions
quiz_questions['question_text'] = quiz_questions['question_text'].apply(lambda x: str(html.fromstring(x).text_content()))

In [92]:
exit_ticket_question_info = quiz_questions[['id', 'quiz_id', 'position', 'question_text', 'answers']]

In [93]:
exit_ticket_question_info

,id,quiz_id,position,question_text,answers
0,103049,15589,1,Which of the following data formats am I most ...,"[{'weight': 0, 'migration_id': 'RESPONSE_8305'..."
1,103050,15589,2,What is the purpose of a status code (like 200...,"[{'weight': 100, 'migration_id': 'RESPONSE_246..."
2,103051,15589,3,What does a '?' in a URL indicate?,"[{'weight': 0, 'migration_id': 'RESPONSE_2000'..."
3,99340,15589,4,This lecture was...,"[{'weight': 100, 'migration_id': 'RESPONSE_468..."
4,99341,15589,5,This lecture was...,"[{'weight': 100, 'migration_id': 'RESPONSE_579..."
5,99342,15589,6,What did you like best about this lecture?,[]
6,99343,15589,7,Do you have any thoughts about how this lectur...,[]
7,99344,15589,8,Is there any other feedback you have about thi...,[]


# Query Submissions for One Quiz

In [52]:
api = 'https://learning.flatironschool.com/api/v1'
course_id = 4142
quiz_id = 15563
endpoint = '/courses/{}/quizzes/{}/submissions'.format(course_id, quiz_id)

url = api + endpoint
print(url)

headers = {
    'Authorization': 'Bearer ' + ACCESS_TOKEN
}

url_params = {
    'offset': 0
}
response = requests.get(url, headers=headers, params=url_params)
print(response.status_code)

https://learning.flatironschool.com/api/v1/courses/4142/quizzes/15563/submissions
200


In [53]:
quiz_submissions = pd.DataFrame(response.json()['quiz_submissions'])
quiz_submissions.head()

,id,quiz_id,quiz_version,user_id,submission_id,score,kept_score,started_at,end_at,finished_at,...,manually_unlocked,validation_token,score_before_regrade,has_seen_results,time_spent,attempts_left,overdue_and_needs_submission,excused?,html_url,result_url
0,71089,15563,7,5283,None,4.0,4.0,2021-05-03T16:20:56Z,None,2021-05-03T16:22:14Z,...,None,56aff47a53bfdfed95a0641cde1fea229ec35a781b2d3f...,None,False,78,0,False,None,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/courses/41...
1,71091,15563,7,5073,None,4.0,4.0,2021-05-03T16:21:05Z,None,2021-05-03T16:29:45Z,...,None,19ae8278f45e23c74ead759e78daa2bd1e69aa19467987...,None,False,520,0,False,None,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/courses/41...
2,71094,15563,7,5612,None,4.0,4.0,2021-05-03T16:21:58Z,None,2021-05-03T16:39:28Z,...,None,692793ffdbb07e207953f654fd2a83c537db5671a9e5ad...,None,False,1050,0,False,None,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/courses/41...
3,71110,15563,7,5194,None,4.0,4.0,2021-05-03T17:01:14Z,None,2021-05-03T17:06:00Z,...,None,3dc26eaa758c28d230c551566c21a07a12b359a20eff1a...,None,False,286,0,False,None,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/courses/41...
4,71166,15563,7,5526,None,3.0,3.0,2021-05-03T19:22:34Z,None,2021-05-03T19:23:40Z,...,None,4968efc6f00bb8a5d419896568386f988cf575c125167a...,None,False,66,0,False,None,https://learning.flatironschool.com/courses/41...,https://learning.flatironschool.com/courses/41...


# Quiz Submission Questions

In [54]:
api = 'https://learning.flatironschool.com/api/v1'
course_id = 4142
quiz_id = 15563
quiz_submission_id = 71166
endpoint = '/quiz_submissions/{}/questions'.format(quiz_submission_id)

url = api + endpoint
print(url)

headers = {
    'Authorization': 'Bearer ' + ACCESS_TOKEN
}

url_params = {
    'offset': 0
}
response = requests.get(url, headers=headers, params=url_params)
print(response.status_code)

https://learning.flatironschool.com/api/v1/quiz_submissions/71166/questions
200


In [64]:
quiz_submission_questions = pd.DataFrame(response.json()['quiz_submission_questions']).sort_values('position')
quiz_submission_questions

,id,quiz_id,quiz_group_id,assessment_question_id,position,question_name,question_type,question_text,answers,variables,formulas,answer_tolerance,formula_decimal_places,matches,flagged,correct
5,103009,15563,None,NaN,1,Question 1,multiple_choice_question,"<span style=""font-size: 18pt;"">Suppose I have ...","[{'id': 3061, 'text': 'Comparing the average l...",None,None,None,None,None,False,True
6,103010,15563,None,NaN,2,Question 2,multiple_choice_question,"<span style=""font-size: 18pt;"">Under what cond...","[{'id': 2282, 'text': 'when I want to stack mu...",None,None,None,None,None,False,True
7,103011,15563,None,116695.0,3,Question 3,multiple_choice_question,"<span style=""font-size: 18pt;"">What does the a...","[{'id': 5964, 'text': 'how many columns to use...",None,None,None,None,None,False,True
0,99182,15563,None,114430.0,4,Question 4,multiple_choice_question,"<span style=""font-size: 18pt;"">This lecture wa...","[{'id': 50332, 'text': 'Very engaging'}, {'id'...",None,None,None,None,None,False,False
1,99183,15563,None,114432.0,5,Question 5,multiple_choice_question,"<span style=""font-size: 18pt;"">This lecture wa...","[{'id': 33385, 'text': 'Very helpful for my le...",None,None,None,None,None,False,False
2,99184,15563,None,114434.0,6,Question 6,essay_question,"<span style="" font-size: 18pt;"">What did you l...",NaN,None,None,None,None,None,False,undefined
3,99185,15563,None,114436.0,7,Question 7,essay_question,"<span style=""font-size: 18pt;""><span>Do you ha...",NaN,None,None,None,None,None,False,undefined
4,99186,15563,None,114428.0,8,Question 8,essay_question,"<span style=""font-size: 18pt;""><span>Is there ...",NaN,None,None,None,None,None,False,undefined
